In [130]:
from bs4 import BeautifulSoup
import pandas as pd
import re # for years < 2009
from math import nan

# Infos

* En 2015: 1ère boucle ¦2ème boucle ¦3ème boucle ¦  temps net ¦

* En 2014:  temps net ¦1ère boucle ¦2ème boucle ¦3ème boucle ¦

* En 2013:  temps net ¦1ère boucle ¦2ème boucle ¦3ème boucle ¦

In [148]:
year = 2006

fp = open('Escaladelite_H_{}.html'.format(year), 'r')
content = fp.read()

In [149]:
document = BeautifulSoup(content, "html.parser")

In [150]:
h3 = document.find('h3')
fonts = h3.findNextSiblings()
if year < 2009:
    content = document.find('div', {'id':'scrapeme'})
    print(content)
else:
    content = fonts[2]
    len(fonts)

<div id="scrapeme">    1. KOGO MICAH                     86 KENYA                         20.02,0       -----    (61)    2.45 ¦     19.58,2 ¦   6.32     1.¦   6.51     1.¦   6.38     1.¦
    2. WEIDLINGER GUENTHER            78 AUTRICHE                      20.02,1      0.00,1    (43)    2.45 ¦     19.58,4 ¦   6.32     1.¦   6.51     1.¦   6.38     2.¦
    3. CHENGERE TOLOSSA               83 ETHIOPIE                      20.25,3      0.23,3     (4)    2.49 ¦     20.21,5 ¦   6.32     4.¦   6.54     3.¦   6.58     4.¦
    4. WYATT JONATHAN                 72 NOUVELLE-ZELANDE              20.36,1      0.34,1    (26)    2.50 ¦     20.31,4 ¦   6.38    13.¦   7.02     4.¦   6.55     3.¦
    5. BELZ CHRISTIAN                 74 SUISSE                        20.42,8      0.40,8    (44)    2.51 ¦     20.38,7 ¦   6.35     7.¦   7.05     6.¦   7.01     6.¦
    6. LEMMA KLEYESSA FEDAKU          84 ETHIOPIE                      20.44,1      0.42,1    (65)    2.51 ¦     20.40,1 ¦   6.32     3.¦   7

In [151]:
content.text
#lines = content.text.split("\n")
elements = content.contents
if year < 2009:
    elements = content.contents[0].split("\n")
len(elements)



601

In [152]:
if year < 2009:
    all_elements = elements
else:
    all_elements = list(content.stripped_strings)
len(all_elements)

601

In [153]:
all_elements[:20]

['    1. KOGO MICAH                     86 KENYA                         20.02,0       -----    (61)    2.45 ¦     19.58,2 ¦   6.32     1.¦   6.51     1.¦   6.38     1.¦',
 '    2. WEIDLINGER GUENTHER            78 AUTRICHE                      20.02,1      0.00,1    (43)    2.45 ¦     19.58,4 ¦   6.32     1.¦   6.51     1.¦   6.38     2.¦',
 '    3. CHENGERE TOLOSSA               83 ETHIOPIE                      20.25,3      0.23,3     (4)    2.49 ¦     20.21,5 ¦   6.32     4.¦   6.54     3.¦   6.58     4.¦',
 '    4. WYATT JONATHAN                 72 NOUVELLE-ZELANDE              20.36,1      0.34,1    (26)    2.50 ¦     20.31,4 ¦   6.38    13.¦   7.02     4.¦   6.55     3.¦',
 '    5. BELZ CHRISTIAN                 74 SUISSE                        20.42,8      0.40,8    (44)    2.51 ¦     20.38,7 ¦   6.35     7.¦   7.05     6.¦   7.01     6.¦',
 '    6. LEMMA KLEYESSA FEDAKU          84 ETHIOPIE                      20.44,1      0.42,1    (65)    2.51 ¦     20.40,1 ¦   6.32     3.¦ 

In [11]:
all_elements[1]

'MBISHEI TITUS'

In [154]:
counter = 0
runners = []

if year < 2009:
    reg_name = re.compile('^\s+\d+\.\s(.*?)\s{2,}')
    reg_time = re.compile('.*?¦(.*?)¦')
    for line in all_elements:
        name_match = re.match(reg_name, line)
        if name_match:
            name = name_match.group(1).title()
        else:
            print('Error with this line', line)
        # temps net = Â¦     **20.21,5** Â¦
        time_match = re.match(reg_time, line)
        if time_match:
            time = time_match.group(1).strip()
        else:
            time = nan
        runners.append({'nom':name, 'temps net':time})
else:
    while counter < len(all_elements)-4:   
        try:
            if year < 2010: # names in all caps
                name = all_elements[counter + 1].title()
            else:
                name = all_elements[counter + 1]
            if year < 2010:
                temps = all_elements[counter + 2].split('¦')
                runners.append({'nom':name, 'temps net':temps[1]})            
            else:
                temps = all_elements[counter + 4].split('¦')
                if year == 2015:
                    runners.append({'nom':name, '1ère boucle':temps[1], '2ème boucle':temps[2], '3ème boucle':temps[3], 'temps net':temps[4]})
                else:
                    runners.append({'nom':name, 'temps net':temps[1], '1ère boucle':temps[2], '2ème boucle':temps[3], '3ème boucle':temps[4]})
        except ValueError as e:
            print('Error with {}: {}'.format(name, e))
        print("Counter = {}, name = {}, temps net = {}".format(counter, name, temps[1]))
        if year < 2010:
            counter += 2
        else:
            counter += 4
runners

Error with this line 


[{'nom': 'Kogo Micah', 'temps net': '19.58,2'},
 {'nom': 'Weidlinger Guenther', 'temps net': '19.58,4'},
 {'nom': 'Chengere Tolossa', 'temps net': '20.21,5'},
 {'nom': 'Wyatt Jonathan', 'temps net': '20.31,4'},
 {'nom': 'Belz Christian', 'temps net': '20.38,7'},
 {'nom': 'Lemma Kleyessa Fedaku', 'temps net': '20.40,1'},
 {'nom': 'Baaru Philemon', 'temps net': '20.44,1'},
 {'nom': 'Wayessa Urguessa', 'temps net': '20.53,6'},
 {'nom': 'Eticha Tesfaye', 'temps net': '20.58,1'},
 {'nom': 'Keitany Elijah', 'temps net': '20.58,9'},
 {'nom': 'Tadese Abraham', 'temps net': '21.05,2'},
 {'nom': 'Kipchilat Philemon', 'temps net': '21.05,2'},
 {'nom': 'Kibet Collins', 'temps net': '21.08,2'},
 {'nom': 'Tanui Mike', 'temps net': '21.09,3'},
 {'nom': 'Kemboi Hillary', 'temps net': '21.12,8'},
 {'nom': 'Andreiev Grigoriy', 'temps net': '21.13,8'},
 {'nom': 'Lidetu Dejene', 'temps net': '21.14,6'},
 {'nom': 'Kadi Nesevo', 'temps net': '21.17,0'},
 {'nom': 'Manirakiza Egide', 'temps net': '21.22,1'},


In [142]:
#runners[-20:]
runners[:20]

[{'nom': 'Chengere Tolossa', 'temps net': '20.16,9'},
 {'nom': 'Kigen Mike', 'temps net': '20.20,5'},
 {'nom': 'Weidlinger Guenther', 'temps net': '20.21,6'},
 {'nom': 'Koech Nicholas', 'temps net': '20.42,7'},
 {'nom': 'Wayessa Urguessa', 'temps net': '21.05,9'},
 {'nom': 'Roethlin Viktor', 'temps net': '21.12,0'},
 {'nom': 'Schaffner Jerome', 'temps net': '21.15,0'},
 {'nom': 'Eticha Tesfaye', 'temps net': '21.16,1'},
 {'nom': 'Bandi Philipp', 'temps net': '21.18,2'},
 {'nom': 'Solomon Tesfamariam', 'temps net': '21.21,9'},
 {'nom': 'Foerster Jan', 'temps net': '21.33,1'},
 {'nom': 'Manirakiza Egide', 'temps net': '21.36,6'},
 {'nom': 'Sekletov Alexandre', 'temps net': '21.40,8'},
 {'nom': 'Kiptum Daniel', 'temps net': '21.41,9'},
 {'nom': 'Khelifi Mehdi', 'temps net': '21.51,4'},
 {'nom': 'Kadi Nesero', 'temps net': '21.58,6'},
 {'nom': 'Tadese Abraham', 'temps net': '22.06,3'},
 {'nom': 'Staehli Stephen', 'temps net': '22.07,1'},
 {'nom': 'Voegeli Daniel', 'temps net': '22.09,6'},


In [155]:
df = pd.DataFrame(runners)
df.head(10)

,nom,temps net
0,Kogo Micah,"19.58,2"
1,Weidlinger Guenther,"19.58,4"
2,Chengere Tolossa,"20.21,5"
3,Wyatt Jonathan,"20.31,4"
4,Belz Christian,"20.38,7"
5,Lemma Kleyessa Fedaku,"20.40,1"
6,Baaru Philemon,"20.44,1"
7,Wayessa Urguessa,"20.53,6"
8,Eticha Tesfaye,"20.58,1"
9,Keitany Elijah,"20.58,9"


In [156]:
df.to_csv('runners_{}.csv'.format(year))